In [0]:
from pyspark.sql.functions import *

### Work On Movie Table

In [0]:
movie_df = spark.sql("""SELECT 
    *, 
    CASE 
        WHEN genres LIKE '%Comedy%' THEN  'Yes'
        ELSE 'No' 
    END AS Comedy,
    CASE
        WHEN genres LIKE '%Action%' THEN 'Yes'
        ELSE 'No' 
    END AS AAction,
    CASE
        WHEN genres LIKE '%Adventure%' THEN 'Yes'
        ELSE 'No' 
    END AS Adventure,
    CASE
        WHEN genres LIKE '%Animation%' THEN 'Yes'
        ELSE 'No' 
    END AS Animation,
    CASE
        WHEN genres LIKE '%Children%' THEN 'Yes'
        ELSE 'No' 
    END AS Children,
    CASE
        WHEN genres LIKE '%Fantasy%' THEN 'Yes'
        ELSE 'No' 
    END AS Fantasy,
    CASE
        WHEN genres LIKE '%Romance%' THEN 'Yes'
        ELSE 'No' 
    END AS Romance,
    CASE
        WHEN genres LIKE '%Sci-Fi%' THEN 'Yes'
        ELSE 'No' 
    END AS SciFi,
    CASE
        WHEN genres LIKE '%Drama%' THEN 'Yes'
        ELSE 'No' 
    END AS Drama,
    CASE
        WHEN genres LIKE '%Thriller%' THEN 'Yes'
        ELSE 'No' 
    END AS Thriller,
    CASE
        WHEN genres LIKE '%Horror%' THEN 'Yes'
        ELSE 'No' 
    END AS Horror,
    CASE
        WHEN genres LIKE '%War%' THEN 'Yes'
        ELSE 'No' 
    END AS War,
    CASE
        WHEN genres LIKE '%Musical%' THEN 'Yes'
        ELSE 'No' 
    END AS Musical,
    CASE
        WHEN genres LIKE '%Documentary%' THEN 'Yes'
        ELSE 'No' 
    END AS Documentary,
    CASE
        WHEN genres LIKE '%Crime%' THEN 'Yes'
        ELSE 'No' 
    END AS Crime 
FROM 
    netflix_data.cleandata.c_movies_table""")


In [0]:
movie_df = movie_df.drop("genres")\
    .withColumnRenamed("title", "Movie_Title")\
    .withColumnRenamed("movieId", "Movie_ID")

In [0]:
movie_df.display()

**Save As Table**

In [0]:
movie_df.write.format('delta')\
              .mode('overwrite')\
              .saveAsTable('netflix_data.transformed_data.t_movies_table')

In [0]:
%sql
SELECT * FROM netflix_data.transformed_data.t_movies_table
WHERE Comedy = 'Yes' AND Children = 'Yes' AND Adventure = 'Yes' AND Animation = 'Yes' AND Fantasy = 'Yes' 

### Work With Ratings

In [0]:
df_avg_rating = spark.sql("""SELECT userId, movieId, round(AVG(rating), 2) AS average_rating
FROM netflix_data.cleandata.c_ratings_table
GROUP BY userId, movieId
ORDER BY userID ASC""")


In [0]:
df_avg_rating.write.format('delta')\
              .mode('overwrite')\
              .saveAsTable('netflix_data.transformed_data.t_avg_rating')

### Join Data Frames

In [0]:
full_df = spark.sql("""SELECT ar.userId, tm.Movie_ID, tm.Movie_Title, ar.average_rating,  tm.Comedy, tm.AAction, tm.Adventure, tm.Animation, tm.Children, tm.Fantasy, tm.Romance, tm.SciFi, tm.Drama, tm.Thriller, tm.Horror, tm.War, tm.Musical, tm.Documentary, tm.Crime, ct.tag, ct.tags_timestamp FROM netflix_data.transformed_data.t_movies_table tm 
               LEFT JOIN netflix_data.transformed_data.t_avg_rating ar
               ON tm.Movie_ID = ar.movieId
               LEFT JOIN netflix_data.cleandata.c_tags_table ct
               ON tm.Movie_ID = ct.movieId
               ORDER BY ar.userId ASC""")

In [0]:
full_df = (full_df.withColumn('userId', when(col('userId').isNull(), 0).otherwise(col('userId')))
    .withColumn('Movie_ID', when(col('Movie_ID').isNull(), 0).otherwise(col('Movie_ID')))
    .withColumn('Movie_Title', when(col('Movie_Title').isNull(), 'Unknown').otherwise(col('Movie_Title')))
    .withColumn('average_rating', when(col('average_rating').isNull(), 0).otherwise(col('average_rating')))
    .withColumn('tag', when(col('tag').isNull(), 'None').otherwise(col('tag')))
    .withColumn('tags_timestamp', when(col('tags_timestamp').isNull(), '1900-01-01 00:00:00').otherwise(col('tags_timestamp')))
    .withColumnRenamed('userId', 'UserID')
    .withColumnRenamed('Movie_ID', 'MovieID')
    .withColumnRenamed('Movie_Title', 'MovieTitle')
    .withColumnRenamed('average_rating', 'AverageRatingOfMovie')
    .withColumnRenamed('AAction', 'Action')
    .withColumnRenamed('tag', 'MovieTag')
    .withColumnRenamed('tags_timestamp', 'MovieTagDateTime'))

In [0]:
full_df = full_df.drop_duplicates(subset=['UserID', 'MovieID', 'MovieTitle'])

In [0]:
full_df.write.format('delta')\
              .mode('overwrite')\
              .saveAsTable('netflix_data.transformed_data.t_full_df')

In [0]:
%sql
SELECT * FROM netflix_data.transformed_data.t_full_df
ORDER BY UserID ASC